In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
# import pytorch_DIW_scratch
import torch.nn as nn
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
# from blurgenerator import motion_blur, lens_blur, gaussian_blur
from PIL import Image
import torchvision.transforms as T
top = T.ToPILImage()
tot = T.ToTensor()

from lens_blur import lens_blur

/home/ubuntu/anaconda3/envs/max_2_0/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
""" Full assembly of the parts to form the complete network """

from unet_parts import *


class UNet(nn.Module):
    def __init__(self, n_channels_org, n_channels_depth, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels_org = n_channels_org
        self.n_channels_depth = n_channels_depth
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc_org = (DoubleConv(n_channels_org, 32))
        self.inc_depth = (DoubleConv(n_channels_depth, 32))

        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))

        self.outc_0 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False), # 3, 32
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False), # 3, 32
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.outc = (OutConv(64, n_classes))

    def forward(self, depth, org):
        d1 = self.inc_depth(depth)
        org1 = self.inc_org(org)

        x1 = torch.cat([d1, org1], axis=1) # (b, 64, w, h)

        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc_0(x)
        x = self.outc(x)

        x = F.softmax(x, dim=1)
        return x

model = UNet(3, 1, 4).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
sum([np.prod(p.size()) for p in model_parameters])


4199940

In [5]:
ipt_depth = torch.rand([1, 1, 384, 512]).cuda()
ipt_image = torch.rand([1, 3, 384, 512]).cuda()

# # bokehNN.eval()
res = model.forward(ipt_depth, ipt_image)



torch.Size([1, 4, 384, 512])

In [7]:

# https://github.com/Soester10/Bokeh-Rendering-with-Vision-Transformers/blob/main/train.py

import PIL.Image as pil
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn as nn
import torch.optim as optim

# from pytorch_msssim.pytorch_msssim import msssim
# from pytorch_ssim.pytorch_ssim import ssim
# import PerceptualSimilarity.lpips.lpips as lpips

from tqdm import tqdm

# feed_width = 768
# feed_height =  512

# feed_height = 384
# feed_width = 512
# feed_height = 768
# feed_width = 1024

feed_height = 1024
feed_width = 1536

batch_size = 1#0


class bokehDataset(Dataset):
    
    def __init__(self, csv_file,root_dir, transform=None):
        
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.root_dir = root_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        bok = pil.open(self.root_dir + self.data.iloc[idx, 0][1:]).convert('RGB')
        org = pil.open(self.root_dir + self.data.iloc[idx, 1][1:]).convert('RGB')


        depth_path = str(self.root_dir + self.data.iloc[idx, 1][1:]
                         ).replace('original', 'depth').replace('jpg', 'png')
        depth = pil.open(depth_path)

        blur_6_path = str(self.root_dir + self.data.iloc[idx, 0][1:]).replace('bokeh', 'lens_blur_6_3_2')
        blur_17_path = str(self.root_dir + self.data.iloc[idx, 0][1:]).replace('bokeh', 'lens_blur_17_3_2')
        blur_65_path = str(self.root_dir + self.data.iloc[idx, 0][1:]).replace('bokeh', 'lens_blur_65_3_3')

# [0.2048, 0.1720, 0.0290, 0.0624, 0.2019, 0.1071, 0.0460, 0.0173, 0.1592, 0.0002]
#    org      6      10      10      16      20      30      50     65     100
        blur_6_path = pil.open(blur_6_path).convert('RGB')

        blur_17_path = pil.open(blur_17_path).convert('RGB')

        blur_65_path = pil.open(blur_65_path).convert('RGB')



        bok = bok.resize((feed_width, feed_height), pil.LANCZOS)
        org = org.resize((feed_width, feed_height), pil.LANCZOS)
        depth = depth.resize((feed_width, feed_height), pil.LANCZOS)

        blur_6_path = blur_6_path.resize((feed_width, feed_height), pil.LANCZOS)
        blur_17_path = blur_17_path.resize((feed_width, feed_height), pil.LANCZOS)
        blur_65_path = blur_65_path.resize((feed_width, feed_height), pil.LANCZOS)

        # blur_25 = Image.fromarray(lens_blur(np.array(org), radius=5, components=5, exposure_gamma=4))


        if self.transform : 
            bok_dep = self.transform(bok)
            org_dep = self.transform(org)
            depth_dep = self.transform(depth)

            blur_6_path = self.transform(blur_6_path)

            blur_17_path = self.transform(blur_17_path)

            blur_65_path = self.transform(blur_65_path)
        # depth_dep = depth_dep / depth_dep.max()
        
        stacked_10 = torch.stack([org_dep, blur_6_path, blur_17_path, blur_65_path], dim=0)

        return (bok_dep, org_dep, depth_dep, stacked_10) # here

transform1 = transforms.Compose([
    transforms.ToTensor(),
])


transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor(),
]) 


transform3 = transforms.Compose([
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor(),
])

trainset1 = bokehDataset(csv_file = '../MegaDepth/Bokeh_Data/train.csv', root_dir = '.',transform = transform1)
trainset2 = bokehDataset(csv_file = '../MegaDepth/Bokeh_Data/train.csv', root_dir = '.',transform = transform2)
trainset3 = bokehDataset(csv_file = '../MegaDepth/Bokeh_Data/train.csv', root_dir = '.',transform = transform3)

trainloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([trainset1,trainset2,trainset3]), batch_size=batch_size,
                                          shuffle=True, num_workers=7)
# trainloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([trainset1]), batch_size=batch_size,
#                                           shuffle=True, num_workers=4)

testset = bokehDataset(csv_file = '../MegaDepth/Bokeh_Data/test.csv',  root_dir = '.', transform = transform1)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=7)


In [8]:
for i, data in enumerate(tqdm(testloader), 0):
    a, b, depth, stacked_10 = data

    print(a.shape, b.shape, depth.shape, stacked_10.shape)
    break


  0%|          | 0/294 [00:00<?, ?it/s]

torch.Size([1, 3, 1024, 1536]) torch.Size([1, 3, 1024, 1536]) torch.Size([1, 1, 1024, 1536]) torch.Size([1, 4, 3, 1024, 1536])


  0%|          | 0/294 [00:02<?, ?it/s]


In [9]:
device = 'cuda'
learning_rate = 0.00001
                                                  
MSE_LossFn = nn.MSELoss()
L1_LossFn = nn.L1Loss()

# state_dict = torch.load('./weights/run_1_12_0.pt')
run_name = '0'
start_ep = 0

# model = torch.compile(model)
# bokehNN = bokehNet(netG).cuda()
# bokehNN.load_state_dict(state_dict)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))


In [10]:
import wandb

id="Unet_lens_4_lpips_finetune_2_continue_50"
import lpips
loss_fn_alex = lpips.LPIPS(net='alex').cuda()

wandb.init(
    project="diploma",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.00001
    },
    # tags=['JPEG/retrain_from_old_jpeg'],
    resume=True,
    id=id
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


wandb: Currently logged in as: maxkuz. Use `wandb login --relogin` to force relogin


Loading model from: /home/ubuntu/anaconda3/envs/max_2_0/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [11]:
parameters = torch.load('/home/ubuntu/max/image/JPEG/C++_pipeline/zip/depth_estimation/DPT/weights/top_4_blurs_finetune/56.pt', map_location=torch.device("cpu"))
model.load_state_dict(parameters)

<All keys matched successfully>

In [13]:
wandb.watch(model, log_freq=15, log='all')


def train(dataloader):
    model.train()

    running_loss = 0

    for i,data in enumerate(tqdm(dataloader), 0):
        if i > 6000:
            break
        bok, org, depth, stacked_10 = data
        bok, org, depth = bok.to(device) , org.to(device), depth.to(device, dtype=torch.float)
        stacked_10 = stacked_10.to(device)
        
        optimizer.zero_grad()

        # bok_mask = model.forward(depth)
        bok_mask = model.forward(depth, org)

        # stacked_images = torch.stack([org, blur_25, blur_45, blur_75], dim=1).cuda()  # [batch_n, 4, 3, h, w]
        # stacked_images = torch.stack([org, blur_45], dim=1).cuda()  # [batch_n, 4, 3, h, w]

        # Multiply the maps and images using broadcasting
        bok_mask = bok_mask.unsqueeze(2)

        output_images = torch.mul(bok_mask, stacked_10) # [batch_n, 4, 3, h, w]
        bok_pred = torch.sum(output_images, dim=1) # [batch_n, 3, h, w]

        # loss_lpips = loss_fn_alex(bok_pred, bok).sum()
        # loss_ssim = 1-ssim(bok_pred, bok)

        loss = (1-ms_ssim(bok_pred, bok))
        # loss_l1 = L1_LossFn(bok_pred, bok)
        # loss = MSE_LossFn(bok_pred, bok)

        # loss = loss_lpips * 1 + loss_l1 * 1.7 + loss_ssim * 35
        running_loss += loss

        loss.backward()
        optimizer.step()
    
        if (i % 500 == 0):
            original = wandb.Image(org[0])
            pred = wandb.Image(bok_pred[0])
            ground_truth = wandb.Image(bok[0])
            wandb.log({
                "Train/1_ms_ssim": loss,
                "Train/loss": loss,
                "Train/orig": original,
                "Train/pred": pred,
                "Train/bokeh": ground_truth,
                "Train/depth": wandb.Image(depth[0] / depth[0].max())
                })
        #     print ('Batch: ',i,'/',len(dataloader),' Loss:', loss.item())

    wandb.log({
        "Train/running_loss": running_loss/len(dataloader)})



def val(dataloader, e):
    running_l1_loss = 0
    running_ms_loss = 0
    running_lpips_loss = 0

    with torch.no_grad():
        avg_w = torch.zeros((4))
        for i,data in enumerate(tqdm(dataloader), 0):

            bok, org, depth, stacked_10 = data
            bok, org, depth = bok.to(device) , org.to(device), depth.to(device, dtype=torch.float)
            stacked_10 = stacked_10.to(device)

            # bok_mask = model.forward(depth)
            bok_mask = model.forward(depth, org)

            # stacked_images = torch.stack([org, blur_25, blur_45, blur_75], dim=1).cuda()  # [batch_n, 4, 3, h, w]
            # stacked_images = torch.stack([org, blur_45], dim=1).cuda()  # [batch_n, 4, 3, h, w]


            # Multiply the maps and images using broadcasting
            bok_mask = bok_mask.unsqueeze(2)
            output_images = torch.mul(bok_mask, stacked_10) # [batch_n, 4, 3, h, w]
            bok_pred = torch.sum(output_images, dim=1) # [batch_n, 3, h, w]

            # l1_loss = L1_LossFn(bok_pred, bok)

            l1_loss = L1_LossFn(bok_pred, bok)
            ms_loss = ssim(bok_pred * 255, bok * 255)
            lpips_loss = loss_fn_alex(bok_pred, bok).sum()

            
            avg_w += torch.tensor((bok_mask[:, 0, :, :, :].mean(),
                bok_mask[:, 1, :, :, :].mean(),
                bok_mask[:, 2, :, :, :].mean(),
                bok_mask[:, 3, :, :, :].mean(),
                ))
                

            if i % 10 == 0:

                original = wandb.Image(org[0])
                pred = wandb.Image(bok_pred[0])
                ground_truth = wandb.Image(bok[0])

                wandb.log({
                    "Val/L1_loss": l1_loss,
                    "Val/lpips_loss": lpips_loss,
                    "Val/orig": original,
                    "Val/pred": pred,
                    "Val/bokeh": ground_truth,
                    "Val/depth": wandb.Image(depth[0]),
                    'Val/map': wandb.Image(bok_mask[0]),
                    # 'Val/blur_25': wandb.Image(blur_25[0]),
                    # 'Val/blur_45': wandb.Image(blur_45[0]),
                    # 'Val/blur_75': wandb.Image(blur_75[0])
                    })
            if i % 30 == 0:
                names = ['org', 'blur_6', 'blur_17', 'blur_65']

                for i in range(1, len(names)):
                    wandb.log({
                        f"Val/{names[i]}": wandb.Image(stacked_10[0][i])
                        }
                    )

            running_l1_loss += l1_loss.item()
            running_ms_loss += ms_loss.item()
            running_lpips_loss += lpips_loss.item()

    wandb.log({
        "Val/L1_loss": running_l1_loss / len(dataloader),
        "Val/MS_loss": running_ms_loss / len(dataloader),
        "Val/LPIPS_loss": running_lpips_loss / len(dataloader),
        "Val/Epoch": e,
        })

    print ('Validation l1 Loss: ',running_l1_loss/ len(dataloader))   
    print ('Validation lpips Loss: ',running_lpips_loss/ len(dataloader), '\n')
    print ('Validation ms Loss: ',running_ms_loss/ len(dataloader), '\n')
    print("Average weights:", avg_w / len(dataloader))

    torch.save(model.state_dict(), f'./weights/top_4_blurs_finetune/{e}.pt')

start_ep = 57

for epoch in range(start_ep, 500):
    print("\nEpoch:", epoch)

    train(trainloader)

    with torch.no_grad():
        val(testloader, epoch)




Epoch: 57


100%|██████████| 294/294 [03:35<00:00,  1.36it/s]


Validation l1 Loss:  0.04840142234247558
Validation lpips Loss:  0.10234804713122901 

Validation ms Loss:  0.885392712695258 

Average weights: tensor([0.2525, 0.1626, 0.3449, 0.2399])

Epoch: 58


  5%|▍         | 621/13200 [04:11<1:24:57,  2.47it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), f'./weights/top_4_blurs_finetune/{43}.pt')


NameError: name 'torch' is not defined